In [1]:
%env LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1
%env LLM_API_KEY=替换为自己的Qwen API Key，如果不需要评估，则不需要

env: LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1
env: LLM_API_KEY=替换为自己的Qwen API Key，如果不需要评估，则不需要


In [2]:
import os
import pandas as pd
from openai import OpenAI

# 加载数据

In [3]:
expr_version = 'product_v01_flowise_basic_rag'

preprocess_output_dir = os.path.join(os.path.pardir, 'outputs', 'v1_20240713')
expr_dir = os.path.join(os.path.pardir, 'experiments', expr_version)

In [4]:
qa_df = pd.read_excel(os.path.join(preprocess_output_dir, 'question_answer.xlsx'))
prediction_df = qa_df[qa_df['dataset'] == 'test'][['uuid', 'question', 'qa_type', 'answer']].rename(columns={'answer': 'ref_answer'})

# 调用Flowise API

In [5]:
import requests

API_URL = "http://192.168.31.92:3000/api/v1/prediction/8e7a0311-69be-4fee-979c-d57ee3726ceb"
def rag(question):
    payload = {
        "question": question
    }
    response = requests.post(API_URL, json=payload)
    return response.json()

In [19]:
rag('这份报告的发布时间是什么时候')

{'text': '报告日期：2023年12月12日。',
 'question': '这份报告的发布时间是什么时候',
 'chatId': '21ab3135-0ecd-404f-af8c-95d2a37d99ed',
 'chatMessageId': '01ac5f18-447a-4553-84b6-0fb2b1462ad6',
 'isStreamValid': False,
 'sessionId': '21ab3135-0ecd-404f-af8c-95d2a37d99ed'}

# 批量预测

In [7]:
from tqdm.auto import tqdm

answer_dict = {}
for idx, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
    uuid = row['uuid']
    question = row['question']
    raw_resp = rag(question)

    answer_dict[question] = {
        'uuid': uuid,
        'ref_answer': row['ref_answer'],
        'gen_answer': raw_resp['text']
    }

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
prediction_df.loc[:, 'gen_answer'] = prediction_df['question'].apply(lambda q: answer_dict[q]['gen_answer'])

In [12]:
prediction_df.sample(5)

,uuid,question,qa_type,ref_answer,gen_answer
232,50f4fb19-9c21-4f0c-babb-2ddb1c33a6d6,公司债的发行规模变化如何？,detailed,上升了2.5%,2023年前10个月，美国公司债发行规模呈现上升态势，增幅为2.5%。
7,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,如何联系报告的研究小组？,detailed,联系人王有鑫，电话010-66594127，或发送邮件至wangyouxin_hq@bank...,您可以拨打电话010-66594127或发送邮件至wangyouxin_hq@bank-of...
312,c47057d6-35c0-4466-bafd-4ed036030c1d,美国单一家庭房贷整体拖欠率在2023年二季度达到多少？,detailed,1.7%,美国单一家庭房贷整体拖欠率在2023年二季度达到1.7%。
80,0943b71a-7314-4cef-a7d9-682c7ddc9d7f,目前情况下，哪种可能性更高，美国经济软着陆还是衰退？,detailed,“软着陆”发生的可能性要高于衰退,从目前情况看，“软着陆”发生的可能性要高于衰退。
60,774ab320-aba8-45a7-a5f4-791782b46d08,哪些因素影响了欧元区和英国的经济增长？,detailed,就业市场韧性消退、内外部需求回落、通胀和利率水平高企、地缘冲突余波未散,欧洲央行收紧货币政策、通胀高企抑制消费能力、疫情放开的促进效应逐步消减以及全球经济放缓导致出...


# 评估

In [13]:
from langchain_openai import ChatOpenAI

judge_llm = ChatOpenAI(
    api_key=os.environ['LLM_API_KEY'],
    base_url=os.environ['LLM_BASE_URL'],
    model_name='qwen2-72b-instruct',
    temperature=0
)

import time

def evaluate(prediction_df):
    """
    对预测结果进行打分
    :param prediction_df: 预测结果，需要包含问题，参考答案，生成的答案，列名分别为question, ref_answer, gen_answer
    :return 打分模型原始返回结果
    """
    prompt_tmpl = """
你是一个经济学博士，现在我有一系列问题，有一个助手已经对这些问题进行了回答，你需要参照参考答案，评价这个助手的回答是否正确，仅回复“是”或“否”即可，不要带其他描述性内容或无关信息。
问题：
<question>
{{question}}
</question>

参考答案：
<ref_answer>
{{ref_answer}}
</ref_answer>

助手回答：
<gen_answer>
{{gen_answer}}
</gen_answer>
请评价：
    """
    results = []

    for _, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        question = row['question']
        ref_answer = row['ref_answer']
        gen_answer = row['gen_answer']

        prompt = prompt_tmpl.replace('{{question}}', question).replace('{{ref_answer}}', str(ref_answer)).replace('{{gen_answer}}', gen_answer).strip()
        result = judge_llm.invoke(prompt).content
        results.append(result)

        time.sleep(1)
    return results

In [14]:
prediction_df['raw_score'] = evaluate(prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
prediction_df['raw_score'].unique()

array(['是', '否'], dtype=object)

In [16]:
prediction_df['score'] = (prediction_df['raw_score'] == '是').astype(int)

In [17]:
prediction_df['score'].mean()

0.82